##关于协程的知识##

async定义协程对象，一个没有马上执行的程序体
asyncio.get_event_loop() 创建事件循环
再把协程对象丢给事件循环管理运行loop.run_until_complete(async_object)
await  挂起协程等待另一个完成
task   创建一个任务，同意管理 loop.create_task(async_object)

等待结束 loog.run_util_complete(task)

future和task没有本质区别
future_task = asyncio.ensure_future(c) 




In [ ]:
import asyncio

def main():
    async def test1():
        print('1')
        await test2()#等待test2完成后运行
        print('2')

    async def test2():
        print('3')
        print('4')

 

    loop = asyncio.get_event_loop()
    loop.run_until_complete(test1())

#if __name__ == '__main__':
#   main()

##多任务协作

这里的tasks,表示一个任务序列(通常为列表)
可以使用 run_until_complete(asyncio.wait(tasks))
注册多个任务也可以使用run_until_complete(asyncio. gather(*tasks))

在PyCharm中能够正确运行，在jupyter notebook中确报错，因为jupyter已经运行了一个event loop。
    RuntimeError: This event loop is already running。


解决办法具体如下：
1.pip3 install nest_asyncio
2.在协程代码开头，加上如下两行：

#允许协程嵌套
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import asyncio, time
import nest_asyncio

nest_asyncio.apply()

async def do_some_work(i, n): #使用async关键字定义异步函数
  print('任务{}等待: {}秒'.format(i, n))
  await asyncio.sleep(n) #休眠一段时间
  return '任务{}在{}秒后返回结束运行'.format(i, n)

start_time = time.time() #开始时间

loop = asyncio.get_event_loop()
tasks = [asyncio.create_task(do_some_work(1, 2)),
        asyncio.create_task(do_some_work(2, 1)),
        asyncio.create_task(do_some_work(3, 3))]


loop.run_until_complete(asyncio.gather(*tasks))

for task in tasks:
  print('任务执行结果: ', task.result()) 

print('运行时间: ', time.time() - start_time)

![https://pic.rmb.bdstatic.com/bjh/down/71b2022fc45aa8195a1fc73ee3b8f957.png]
试试，当前页所有的详情页地址，和详情页的标题

1、起始地址："https://dp2030k.xyz/pw/thread1022.php?fid=114&page=(页数)"
class="tr3 t_one"
下面第二个 td  h3 a href
<td style="text-align:left;line-height:25px; FONT-SIZE:11pt " id="td_6539219">

			<h3>

			<a href="html_data/114/2303/6539219.html" id="a_ajax_6539219">【PhotoDromm】Mar 16, 2023 - Tessa - The Canyon【46P】</a></h3>&nbsp;  


	</td>
 

In [ ]:
import os 
import string

url = "https://xxxx.com/paht//aaa/bbb\ccc.html?sss=lll"

url=url.replace('\\','/')

ls = url.split('/')
print(ls[0]+"//"+ls[2])

In [ ]:
import os, time
import requests
import asyncio
import nest_asyncio

from lxml import etree, html



nest_asyncio.apply()

headers = ""

start_url = "https://dp2030k.xyz/pw/thread1022.php?fid=114&page=1"  

#工具，找出url的根。形如https://xxxx.com/ 
#相当与 (http|https)://host
def get_url_base(url):
    url=url.replace('\\','/')
    ls = url.split('/')
    return (ls[0]+"//"+ls[2])

#从起始页，找到所有详情页的链接
#返回详情页的标题名字和链接，组成一个list
def get_link_list(start_url):
    try:
        response = requests.get(start_url, headers=headers)
            
    except requests.RequestException as e:
        print (e)
            #print ("状态码：",r.status_code)
            #print ("网站源代码",r.text)
            #print ("头部请求",r.headers) 
        return
    
    urls=[]

    myhtml = etree.HTML(response.content.decode())

    ul = myhtml.xpath("//tr[@class = 'tr3 t_one']")

    for li in ul:
        #print(li)
        block = li.xpath("./*/h3")
        if isinstance( block ,list) and len(block)>=1:
            block = block[0]
        else:
            continue

        bname = block.xpath("./a/text()")
        if len(bname)<1:
            continue

        bname =bname[0] #xpath出来的是个 list 对象。
        burl  = block.xpath("./a/@href")[0]
        if "html_data" not in burl:
             continue
        
        if "http" not in burl:#相对路径，要改成绝对路径
            root_url = get_url_base(start_url)
            burl = root_url+'/pw/'+burl

        #print(burl,": ",bname)
        urls.append([bname,burl])
      
        
    return urls

#get_link_list(start_url)

#在详情页中，读出每个图片的地址
#返回地址列表
def get_image_url(start_url):
    try:
        response = requests.get(start_url, headers=headers)
            
    except requests.RequestException as e:
        print (e)
            #print ("状态码：",r.status_code)
            #print ("网站源代码",r.text)
            #print ("头部请求",r.headers) 
        return
    
    myhtml = etree.HTML(response.content.decode())

    ul = myhtml.xpath("//div[@id = 'read_tpc']")
    if len(ul)<1:
        print("没有找到图床"+start_url)
        return []
    
    ul = ul[0]
    ul = ul.xpath("./img/@src")

    #print(ul)

    return ul

#get_image_url("https://dp2030k.xyz/pw/html_data/114/2303/6539221.html")

#下载图片并且保存在root目录下。
#root目录自动生成目录。
#有时候会很慢，卡住。需要改进。
async def save_img(root, img_url):
    path = root+"/"
    basename = os.path.basename(img_url)
    file_name = path + basename

    #不重复下载
    if True ==os.path.exists(file_name):
        return

    try:
        content = requests.get(img_url, headers=headers,timeout=8).content
    except requests.RequestException as e:
        print (e)
            #print ("状态码：",r.status_code)
            #print ("网站源代码",r.text)
            #print ("头部请求",r.headers) 
        return
    
    
    #print(file_name)

    if not os.path.exists(path):
        os.makedirs(path)
    with open(file_name, 'wb') as f:
        f.write(content)
        f.close()

#root = "【MetArt】Mar 17, 2023 - Euhenia - Classic Glam【99P】"
#img_url = "https://p23.picd232.cc/i/2023/03/18/tybav5.jpg"
#save_img(root,img_url)
if __name__ == '__main__':

    start_url_list=[

        "https://dp2030k.xyz/pw/thread1022.php?fid=220&page=1",
    ]

    #这一步卡住不动了，为什么？
    #溫文爾雅韓 國熟女美模寫真 - 麗雅 (44P): https://dp2030k.xyz/pw/html_data/14/2303/6536850.html

    detail_page_list=[]

    for url in start_url_list:
        ls = get_link_list(url)
        while(len(ls)>0):
            detail_page_list.append(ls.pop())

    image_list =[]
    for page in detail_page_list:
        root ="output/"+page[0]
        url  = page[1]

        print(root+": "+url)
        #continue

        ls = get_image_url(page[1])

        #for img in ls:
        #     save_img(root,img)

        loop = asyncio.get_event_loop()
        tasks=[]
        for img in ls:
            task = asyncio.create_task (save_img(root,img) )
            tasks.append(task)
        #tasks = [save_img(root, img) for img in enumerate(ls)] 
        try:
            #loop.run_until_complete(asyncio.wait(tasks)) 
            loop.run_until_complete(asyncio.gather(*tasks)) 
        finally:
            continue
            #loop.stop()#jupyte会报错，正常环境不会。


In [ ]:
import requests
from lxml import etree, html

headers =  {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36"}
#在详情页中，读出每个图片的地址
#返回地址列表
def get_image_url(start_url):
    try:
        response = requests.get(start_url,headers=headers)
            
    except requests.RequestException as e:
        print (e)
            #print ("状态码：",r.status_code)
            #print ("网站源代码",r.text)
            #print ("头部请求",r.headers) 
        return
    
    myhtml = etree.HTML(response.content.decode())
    
#//form[@id="form"]//span | //form[@id="form"]//input 同时查找多个路径。取或。read_tpc
    ur = myhtml.xpath("//*/img/@src")
    ul = myhtml.xpath("//* [@id='read_tpc']")

    str = response.content.decode()

    substr = str.find("read_tpc")
    substr = str[substr:]

    if len(ul)<1:
        print("没有找到图床"+start_url)
        return []
    
    ul = ul[0]
    ul = ul.xpath("./img/@src")

    #print(ul)

    return ul

#urls= get_image_url("https://dp2030k.xyz/pw/html_data/14/2303/6540567.html")
#print(urls)
urls= get_image_url("https://dp2030k.xyz/pw/html_data/220/2303/6532917.html" )
print(urls)